## 로지스틱 회귀(Logistic Regression) - 심화

선형회귀(Linear Regression) 경우,<br>
공부한 시간(2,3,6,9,11시간)과 기말고사 성적(60,65,70,80,90점)과의 관계를 학습한 결과로<br>
새로운 데이타(13시간)가 들어왔을때 성적을 예측하는 모델로 사용됩니다.<br>

반면에 로지스틱회귀(Rogistic Regression) 경우,<br>
Classification(분류) 문제를 다루는 모델입니다.<br>
맞다 / 틀리다, 살았다 /죽었다, 암이다 / 암이 아니다 ...경우처럼<br>
Target의 카테고리가 2개인 이진 분류에서 주로 사용되는 회귀 모델입니다.<br>

즉, Linear Regression은 연속적인 숫자 선상에 있는 값을 예측할때 사용하는 모델이고<br>
Rogistic Regression은 둘중 하나르 선택할 때 사용하는 모델이라 할 수 있습니다.

### Configuration

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Generate Datasets
로지스틱 회귀를 가장 잘 이해할 수 있는 데이타셋을 예시로 들어보겠습니다.<br>
Feature 생성 - salaries, satisfactions<br>

S전자 신입사원 10명의 (2년후) 연봉 정보를 가져옵니다.<br>
이를 salaries라는 변수에 할당한 후, 행렬 연산을 편하게 하기 위해 Numpy의 배열로 변경해줍니다.<br>
단 여기서 편의를 위해서 단위는 1,000만원으로 하겠습니다.(가령 5.0이면 연봉 5천만원)

#### salaries

In [2]:
salaries = [5.0, 5.5, 5.3, 6.2, 5.25, 5.2, 5.5, 4.9, 5.35, 5.25]

#### satisfactions

S전자 신입사원 10명의 업무 만족도 정보를 가져옵니다.<br>
점수는 1점부터 10점으로 구성되어 있습니다. <br>
이를 satisfactions 라는 변수에 할당한 뒤, 행렬 연산을 편하게 하기 위해서 역시 Numpy의 array() 함수를 사용하겠습니다.

In [4]:
satisfactions = [3, 7, 4, 9, 8, 6, 5, 4, 6, 7]

#### Label생성  -   stay

위에서 2개의 feature를 만들었습니다.<br>
그리고 위 신입사원들의 이직 여부(회사 잔류여부)를 표현한 데이타를 생성합니다.<br>
이 데이타가 오늘 다룰 머신러닝 알고리즘의 Label이 될 것입니다.<br>

S전자 신입사원 10명의 이직 여부(회사 잔류여부)를 저장합니다.<br>
이직하지 않고 회사에 남아있는 경우 True 값을 넣으며, 반대로 다른 회사로 이직했을 경우 False라는 값을 넣습니다.<br>
이 값들을 stay라는 변수에 할당한 후<br>
역시 행렬연산을 위해서 Numpy의 array()를 사용합니다.

In [ ]:
stay = [False, True, False, True, True, True, True, False, True, True]

### Visualize - DataFrame

앞서 생성한 데이타를 하나로 묶어서 표(Table) 로 표현합니다.<br>
그리고 기 결과를 example 이라는 변수에 할당합니다.

### Visualize - Matplotlib

주어진 데이타를 바탕으로 연봉과 업무만족도가 이직에 얼마나 영향을 미치는지 시각화 할수 있습니다.<br>
보라색은 퇴직한 사람.<br>
c는 Color로 이직여부로 구분을 했습니다.<br>

### Define Sigmoid


RogisticRegression 모델에는 Sigmoid 함수를 사용합니다.<br>


이제 로지스틱 회귀(Logistic Regression)를 돌리기 위해 필요한 기능을 구현하겠습니다. 먼저 스쿼싱 함수(Squashing Function)로 사용할 시그모이드(Sigmpod)를 직접 구현해 보겠습니다. 시그모이드 함수의 공식은 다음과 같습니다.

$$\text{sigmoid(x)}  = \frac{1}{1+ e^{-x}} $$

이 공식을 그래프로 시각화 하면 다음과 같이 나올 것입니다.
![pic.jpg](https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/320px-Logistic-curve.svg.png)

이제 위에서 설명한 내용을 바탕으로 시그모이드 함수를 직접 구현해보겠습니다. 시그모이드 함수를 구현하기 위해서는 넘파이(Numpy)에서 지수 함수(exponential fucntion)를 계산하는 ```np.exp```를 사용해야 합니다.

### Gradient Descent - 경사하강법

이제 본격적으로 로지스틱 회귀 알고리즘을 학습시켜 보겠습니다. <br>
먼저 앞서 생성한 두개의 Feature(salaries, satisfactions)를 편의상 각각 x1, x2 라는 변수에 할당합니다.

회사의 잔류 여부를 나타내는 stay를 Label로 간주하고 y변수에 할당

이제 본격적으로 학습을 시작해봅시다. <br>
두 개의 변수 x1과 x2를 가지고 있으니 총 세 개의 값 <br>
바로 가중치(weight) w1, w2과, bias인 b를 학습해야 합니다.<br>
세 개를 변수로 정의한 뒤, 경사 하강법(Gradient Descent)을 이용해 로지스틱 회귀를 학습시켜줍니다.<br>

학습은 모든 epoch이 끝날 때까지 반복할 수도 있지만, <br>
여기서는 굳이 그럴 필요없이 정확도(accuracy)가 1.0에 도달하면 학습을 종료하도록 하겠습니다.<br>

### Predict - 결과 예측하기

로지스틱 회귀의 학습이 끝났으면,<br>
이 머신러닝 알고리즘을 활용하여 원하는 결과를 예측할 수 있습니다.<br>
앞선 데이터를 그대로 활용하여,<br>
S전자의 신입 데이터 사이언티스트들이 2년 후에 이직할 것인지 여부를 예측해 보도록 하겠습니다.<br>